# Fully connected NN on bottleneck features from augmented images
Input is the original plus augmentation features from Xception.

Then train a fully connected NN with 1 hidden layer, last layer with softmax.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import _pickle as pickle
from os import listdir
from os.path import join, isfile

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras import backend as K
K.set_session(session)

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l1, l2, l1_l2

from sklearn.metrics import log_loss, accuracy_score

Using TensorFlow backend.


In [2]:
def add_augmented_labels(ls, inds):
    temp_augs = np.tile(np.arange(NUM_AUGS), (len(inds), 1))
    temp_inds = inds.values.reshape((len(inds), 1)) * NUM_AUGS
    new_inds = np.add(temp_inds, temp_augs).reshape(len(inds)*NUM_AUGS)
    new_ls = ls.reindex(np.repeat(ls.index.values, NUM_AUGS), method='ffill')
    new_ls = new_ls.iloc[new_inds, :]
    return new_ls, new_inds

In [3]:
# Load labels and split to train/val
NUM_CLASSES = 120
SEED = 1993
NUM_AUGS = 10
np.random.seed(seed=SEED)
data_dir = '../data'

labels = pd.read_csv(join(data_dir, 'labels.csv'))
print('Number of all train images: {}'.format(len(labels)))
print("Train data has {} classes.".format(len(labels.groupby('breed').count())))
assert len(labels.groupby('breed').count()) == NUM_CLASSES, 'Number of classes in training set is not 120!'

sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
print('Number of all test images: {}'.format(len(sample_submission)))

# Split to train and validation sets
temp_labels_tr = labels.groupby('breed').apply(pd.DataFrame.sample, frac=0.8)
temp_labels_val = labels.loc[~labels['id'].isin(temp_labels_tr['id'])]
l_tr, inds_tr = add_augmented_labels(labels, temp_labels_tr.index.levels[1])
l_val, inds_val = add_augmented_labels(labels, temp_labels_val.index)

breed_index = {label:i for i,label in enumerate(np.unique(l_tr.breed))}
l_tr_temp = [breed_index[label] for label in l_tr.breed]
l_val_temp = [breed_index[label] for label in l_val.breed]
y_tr = to_categorical(l_tr_temp ,num_classes=120)
y_val = to_categorical(l_val_temp ,num_classes=120)

print('y_tr shape: {}'.format(y_tr.shape))
print('y_val shape: {}'.format(y_val.shape))

Number of all train images: 10222
Train data has 120 classes.
Number of all test images: 10357
y_tr shape: (81850, 120)
y_val shape: (20370, 120)


In [4]:
filename = data_dir + '//train//xs_bf_xception_aug'
print('Loading from {}'.format(filename))
with open(filename, 'rb') as fp:
    xs_bf = pickle.load(fp)
print('xs_bf shape: {} size: {:,}'.format(xs_bf.shape, xs_bf.size))

Loading from ../data//train//xs_bf_xception_aug
xs_bf shape: (102220, 2049) size: 209,448,780


In [5]:
# Split to train/val sets
x_tr = xs_bf.values[inds_tr, :-1]
print('Train bottleneck features shape: {} size: {:,}'.format(x_tr.shape, x_tr.size))

x_val = xs_bf.values[inds_val, :-1]
print('Validation bottleneck features shape: {} size: {:,}'.format(x_val.shape, x_val.size))

Train bottleneck features shape: (81850, 2048) size: 167,628,800
Validation bottleneck features shape: (20370, 2048) size: 41,717,760


In [6]:
# Only original images
x_tr_orig = x_tr[::10]
y_tr_orig = y_tr[::10]
x_val_orig = x_val[::10]
y_val_orig = y_val[::10]
print('Original train bottleneck features shape: {} size: {:,}'.format(x_tr_orig.shape, x_tr_orig.size))

Original train bottleneck features shape: (8185, 2048) size: 16,762,880


In [7]:
# Balance classes with augmented examples such that each class has exactly 101 images
max_class_n = max(y_tr_orig.sum(axis=0)) # 101
topick = [4, 5, 8, 9]
inds_class = []
for i in range(120):
    current_class_n = y_tr_orig.sum(axis=0)[i]
    for j in range(y_tr_orig.shape[0]):
        if np.argmax(y_tr_orig[j]) == i:
            current_class_n = current_class_n + 1
            inds_class.append(10*j + 1)
            if max_class_n <= current_class_n:
                break
x_tr_class = np.concatenate((x_tr_orig, x_tr[inds_class]))
y_tr_class = np.concatenate((y_tr_orig, y_tr[inds_class]))
print('Class balanced train bottleneck features shape: {} size: {:,}'.format(x_tr_class.shape, x_tr_class.size))

Class balanced train bottleneck features shape: (12121, 2048) size: 24,823,808


In [24]:
# Train and evaluate models

# Save predictions on all images into preds variable
preds = []

def setup_model():
    model = Sequential()
    model.add(Dense(200, input_shape=(2048,), kernel_regularizer=l2(0.00001)))
    model.add(Activation('relu'))
    model.add(Dropout(0.6))
    model.add(Dense(120, activation='softmax', kernel_regularizer=l2(0.000001)))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model
 
# Train on original images
model = setup_model()
model.fit(x_tr_orig, y_tr_orig,
          batch_size=512,
          epochs=50,
          verbose=0,
          validation_data=(x_val_orig, y_val_orig),
          shuffle=True,
          callbacks=[EarlyStopping(monitor='val_loss', patience=2)])
scores = model.evaluate(x_val_orig, y_val_orig)
print('Train on original images validation loss: {:.3f} | acc: {:.1%}'.format(scores[0], scores[1]))
preds.append(model.predict(x_val, batch_size=512))

# Train on class balanced images
model = setup_model()
model.fit(x_tr_class, y_tr_class,
          batch_size=512,
          epochs=50,
          verbose=0,
          validation_data=(x_val_orig, y_val_orig),
          shuffle=True,
          callbacks=[EarlyStopping(monitor='val_loss', patience=2)])
scores = model.evaluate(x_val_orig, y_val_orig)
print('Train on class balanced images validation loss: {:.3f} | acc: {:.1%}'.format(scores[0], scores[1]))
preds.append(model.predict(x_val, batch_size=512))

# Train on all images
model = setup_model()
model.fit(x_tr, y_tr,
          batch_size=5120,
          epochs=50,
          verbose=0,
          validation_data=(x_val_orig, y_val_orig),
          shuffle=True,
          callbacks=[EarlyStopping(monitor='val_loss', patience=2)])
scores = model.evaluate(x_val_orig, y_val_orig)
print('Train on all images validation loss: {:.3f} | acc: {:.1%}'.format(scores[0], scores[1]))
preds.append(model.predict(x_val, batch_size=512))

2037/2037 [==============================] - 1s 307us/step
Train on original images validation loss: 0.309 | acc: 90.9%
2037/2037 [==============================] - 0s 238us/step
Train on class balanced images validation loss: 0.316 | acc: 89.3%
2037/2037 [==============================] - 1s 269us/step
Train on all images validation loss: 0.320 | acc: 90.2%


In [52]:
# Calculate loss and accuracy for all models on all images
loss = np.zeros((3, 4))
acc = np.zeros((3, 4))

for i, pred in enumerate(preds):
    
    pred_all = pred
    loss[i][0] = log_loss(y_val, pred_all)
    acc[i][0] = accuracy_score((y_val * range(NUM_CLASSES)).sum(axis=1), np.argmax(pred_all, axis=1))

    pred_orig = pred[::10]
    loss[i][1] = log_loss(y_val_orig, pred_orig)
    acc[i][1] = accuracy_score((y_val_orig * range(NUM_CLASSES)).sum(axis=1), np.argmax(pred_orig, axis=1))
    
    pred_avg = np.average(pred.reshape((int(pred.shape[0]/10), 10, 120)), axis=1)
    pred_avg = pred_avg / np.sum(pred_avg, axis=0)
    loss[i][2] = log_loss(y_val_orig, pred_avg)
    acc[i][2] = accuracy_score((y_val_orig * range(NUM_CLASSES)).sum(axis=1), np.argmax(pred_avg, axis=1))
    
    pred_flip = np.average(pred.reshape((int(pred.shape[0]/10), 10, 120)), axis=1, weights=[0.5, 0.5, 0, 0, 0, 0, 0, 0, 0, 0])
    pred_flip = pred_flip / np.sum(pred_flip, axis=0)
    loss[i][3] = log_loss(y_val_orig, pred_flip)
    acc[i][3] = accuracy_score((y_val_orig * range(NUM_CLASSES)).sum(axis=1), np.argmax(pred_flip, axis=1))

In [53]:
notes = [
    'Validation on all images',
    'Validation on original images only',
    'Validation on all images averaged',
    'Validation on original and flipped image, averaged'
]

print('Validation categorical crossentropy loss:')
print('  Original | Class balanced | Augmented  ')
for i in range(4):
    print('    {:.3f}  |      {:.3f}     |   {:.3f}   <- {}'.format(loss[0][i], loss[1][i], loss[2][i], notes[i]))
print()

print('Validation accuracy:')
print('  Original | Class balanced | Augmented  ')
for i in range(4):
    print('  {:.3%}  |     {:.3%}    |  {:.3%}   <- {}'.format(acc[0][i], acc[1][i], acc[2][i], notes[i]))
print()

Validation categorical crossentropy loss:
  Original | Class balanced | Augmented  
    0.551  |      0.558     |   0.498   <- Validation on all images
    0.300  |      0.308     |   0.311   <- Validation on original images only
    0.384  |      0.396     |   0.381   <- Validation on all images averaged
    0.282  |      0.291     |   0.294   <- Validation on original and flipped image, averaged

Validation accuracy:
  Original | Class balanced | Augmented  
  83.490%  |     82.941%    |  84.831%   <- Validation on all images
  90.869%  |     89.347%    |  90.231%   <- Validation on original images only
  88.807%  |     89.249%    |  89.543%   <- Validation on all images averaged
  90.967%  |     90.280%    |  90.427%   <- Validation on original and flipped image, averaged



In the tables above, the results of 3 models on 4 validation sets are summarized. Columns represent models, rows represent validation image sets. Note that the 1st validation set (on all images) is not very useful, as it cannot be used to make a submission.

Instead of retraining the xception imagenet model weights, we only use the xception model to get the bottleneck features (2048 values of the penultimate layer). My guess is that image augmentation would be much more helpful in the former case, as literature suggests. However, in the results above, the image augmentation does not help. 
(I played with the models manually in other notebooks as well but the non-augmented model was always better)

On the other hand, a pleasant discovery occured. Turns out when evaluating a model, it performs better when evaluated on the average prediction of the original and the horizontally flipped image. 